In [ ]:
import os
arch = os.getenv("ARGS", "real")

In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import petsc4py
else:
    try:
        import petsc4py
    except ImportError:
        if arch != "complex":
            !wget "https://fem-on-colab.github.io/releases/petsc4py-install-real.sh" -O "/tmp/petsc4py-install.sh" && bash "/tmp/petsc4py-install.sh"
        else:
            !wget "https://fem-on-colab.github.io/releases/petsc4py-install-complex.sh" -O "/tmp/petsc4py-install.sh" && bash "/tmp/petsc4py-install.sh"
        import petsc4py

In [ ]:
petsc4py.init()
from petsc4py import PETSc

In [ ]:
import numpy as np
if arch != "complex":
    assert not np.issubdtype(PETSc.ScalarType, np.complexfloating)
else:
    assert np.issubdtype(PETSc.ScalarType, np.complexfloating)

In [ ]:
A = PETSc.Mat().createAIJ([3, 3], nnz=1)
for i in range(3):
    A.setValue(i, i, i + 1)
A.assemble()

In [ ]:
assert np.allclose(A.getValues(range(3), range(3)), np.diag([1., 2., 3.]))

In [ ]:
b = PETSc.Vec().createSeq(3)
b.setValues(range(3), [i + 1 for i in range(3)])

In [ ]:
assert np.allclose(b.getArray(), [1., 2., 3.])

In [ ]:
x = PETSc.Vec().createSeq(3)

In [ ]:
ksp = PETSc.KSP().create()
ksp.setOperators(A)
ksp.solve(b, x)
assert np.allclose(x.getArray(), [1., 1., 1.])

In [ ]:
for package in ("mumps", "superlu", "superlu_dist"):
    ksp = PETSc.KSP().create()
    ksp.setOperators(A)
    ksp.setType("preonly")
    pc = ksp.getPC()
    pc.setType("lu")
    pc.setFactorSolverType(package)
    ksp.solve(b, x)
    assert np.allclose(x.getArray(), [1., 1., 1.])